In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/automatic-ticket-classification/complaints-2021-05-14_08_16_.json


In [39]:
import json

In [40]:
file = open('/kaggle/input/automatic-ticket-classification/complaints-2021-05-14_08_16_.json')
data = json.load(file)
df = pd.json_normalize(data)
df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [41]:
df.shape

(78313, 22)

In [42]:
useless_feat = []
for feat in df.columns:
    if len(df[feat].unique())==1 or len(df[feat].unique()) == len(df):
        useless_feat.append(feat)
useless_feat  

['_index', '_type', '_id', '_score', '_source.complaint_id', '_source.company']

In [43]:
df.drop(useless_feat,axis=1,inplace=True)
df.shape

(78313, 16)

### Aim :- Create a model that segrates the given data into 5 different categories based on compliant type
##### Credit card / Prepaid card
##### Bank account services
##### Theft/Dispute reporting
##### Mortgages/loans
##### Others

##### Here we are have to focus on what are services/products and what is complaint, We dont need all columns to build this model

##### Personal Information doesnot help to classify ticket
######                        '_source.tags','_source.zip_code','_source.state','_source.consumer_disputed','_source.consumer_consent_provided'
##### Datetime,company response,mode of complaint ddoesnot help to classify ticket  
###### '_source.date_received','_source.company_response','_source.submitted_via','_source.date_sent_to_company','_source.company_public_response','_source.timely'

In [44]:
df.drop([ '_source.tags','_source.zip_code','_source.state','_source.consumer_disputed','_source.consumer_consent_provided','_source.date_received','_source.company_response','_source.submitted_via','_source.date_sent_to_company','_source.company_public_response','_source.timely'],axis=1,inplace=True)
df.head()

,_source.issue,_source.product,_source.sub_product,_source.complaint_what_happened,_source.sub_issue
0,Attempts to collect debt not owed,Debt collection,Credit card debt,,Debt is not yours
1,Written notification about debt,Debt collection,Credit card debt,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt
2,"Other features, terms, or problems",Credit card or prepaid card,General-purpose credit card or charge card,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card
3,Trouble during payment process,Mortgage,Conventional home mortgage,,None
4,Fees or interest,Credit card or prepaid card,General-purpose credit card or charge card,,Charged too much interest


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   _source.issue                    78313 non-null  object
 1   _source.product                  78313 non-null  object
 2   _source.sub_product              67742 non-null  object
 3   _source.complaint_what_happened  78313 non-null  object
 4   _source.sub_issue                32016 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [46]:
df= df.mask(df== '')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   _source.issue                    78313 non-null  object
 1   _source.product                  78313 non-null  object
 2   _source.sub_product              67742 non-null  object
 3   _source.complaint_what_happened  21072 non-null  object
 4   _source.sub_issue                32016 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [47]:
df.head()

,_source.issue,_source.product,_source.sub_product,_source.complaint_what_happened,_source.sub_issue
0,Attempts to collect debt not owed,Debt collection,Credit card debt,NaN,Debt is not yours
1,Written notification about debt,Debt collection,Credit card debt,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt
2,"Other features, terms, or problems",Credit card or prepaid card,General-purpose credit card or charge card,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card
3,Trouble during payment process,Mortgage,Conventional home mortgage,NaN,None
4,Fees or interest,Credit card or prepaid card,General-purpose credit card or charge card,NaN,Charged too much interest


In [48]:
import nltk,re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [49]:
stop_words = set(stopwords.words("english"))

In [50]:
'None' in stop_words

False

In [51]:
df.fillna('None',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   _source.issue                    78313 non-null  object
 1   _source.product                  78313 non-null  object
 2   _source.sub_product              78313 non-null  object
 3   _source.complaint_what_happened  78313 non-null  object
 4   _source.sub_issue                78313 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [52]:
df.columns = ['issue','product','sub_product','complaint','sub_issue']
df.head()

,issue,product,sub_product,complaint,sub_issue
0,Attempts to collect debt not owed,Debt collection,Credit card debt,None,Debt is not yours
1,Written notification about debt,Debt collection,Credit card debt,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt
2,"Other features, terms, or problems",Credit card or prepaid card,General-purpose credit card or charge card,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card
3,Trouble during payment process,Mortgage,Conventional home mortgage,None,None
4,Fees or interest,Credit card or prepaid card,General-purpose credit card or charge card,None,Charged too much interest


In [53]:
df = df[['product','sub_product','issue','sub_issue','complaint']]
df.head()

,product,sub_product,issue,sub_issue,complaint
0,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt is not yours,None
1,Debt collection,Credit card debt,Written notification about debt,Didn't receive enough information to verify debt,Good morning my name is XXXX XXXX and I apprec...
2,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Problem with rewards from credit card,I upgraded my XXXX XXXX card in XX/XX/2018 and...
3,Mortgage,Conventional home mortgage,Trouble during payment process,None,None
4,Credit card or prepaid card,General-purpose credit card or charge card,Fees or interest,Charged too much interest,None


In [54]:
df['big_corpus'] = df['product']+' '+df['sub_product']+' '+df['issue']+' '+df['sub_issue']+' '+df['complaint']

In [55]:
df.big_corpus[4]

'Credit card or prepaid card General-purpose credit card or charge card Fees or interest Charged too much interest None'

In [56]:
df = df[['big_corpus']]
df.head()

,big_corpus
0,Debt collection Credit card debt Attempts to c...
1,Debt collection Credit card debt Written notif...
2,Credit card or prepaid card General-purpose cr...
3,Mortgage Conventional home mortgage Trouble du...
4,Credit card or prepaid card General-purpose cr...


In [73]:
import nltk,re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [74]:
ps = PorterStemmer()
ws = WordNetLemmatizer()

In [78]:
parsed = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]',' ',df['big_corpus'][i])
    review = review.lower()
    review = review.split()
    review = [ws.lemmatize(word) for word in review if word not in stop_words]
    review = list(set(review)-set(['xxxx','xxx','xx']))
    parsed.append(" ".join(review))

In [77]:
# nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [79]:
parsed

['owed collection card collect attempt credit none debt',
 'chase verification consumer card advance appreciate validate debt every notification enough statement acceptable information morning written right good thanks verify service sent receiving attempting asking help month mail collect could instead name stop receive cardmember collection bank wrote know put account credit',
 'misled feature purpose upgraded recording date charge would consent prepaid reward card general change without turned anniversary told information agent order wrong term changed giving upgrade account credit problem',
 'process conventional payment mortgage home trouble none',
 'purpose charged interest prepaid general card much credit fee charge none',
 'atm saving debit managing checking card using account none problem',
 'atm saving debit managing checking card using account none problem',
 'process conventional payment mortgage home trouble none',
 'company closed saving closing checking account none',
 '

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=2, max_df=0.95)

In [81]:
tfidf_vectors = tfidf.fit_transform(parsed)
tfidf_vectors

<78313x13526 sparse matrix of type '<class 'numpy.float64'>'
	with 2076220 stored elements in Compressed Sparse Row format>

In [82]:
from sklearn.decomposition import NMF

In [83]:
nmf_model = NMF(n_components=5, random_state=1)

In [84]:
nmf_model.fit(tfidf_vectors)

/opt/conda/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


NMF(n_components=5, random_state=1)

In [90]:
for index,topic in enumerate(nmf_model.components_):
    print(f'Topic {index} top 20 words are ')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[::-1][:30]])
    print('##########################')

Topic 0 top 20 words are 
['modification', 'foreclosure', 'collection', 'mortgage', 'loan', 'none', 'conventional', 'fixed', 'fha', 'adjustable', 'arm', 'home', 'equity', 'line', 'debt', 'credit', 'struggling', 'originator', 'broker', 'pay', 'collect', 'va', 'vehicle', 'owed', 'application', 'attempt', 'lease', 'settlement', 'cont', 'underwriting']
##########################
Topic 1 top 20 words are 
['withdrawal', 'deposit', 'checking', 'saving', 'managing', 'account', 'none', 'service', 'bank', 'problem', 'banking', 'atm', 'debit', 'using', 'product', 'fund', 'error', 'low', 'caused', 'fee', 'closed', 'receiving', 'making', 'company', 'accessing', 'charging', 'lender', 'disbursed', 'instructed', 'handled']
##########################
Topic 2 top 20 words are 
['card', 'credit', 'none', 'prepaid', 'general', 'purpose', 'charge', 'dispute', 'problem', 'billing', 'statement', 'shown', 'purchase', 'company', 'resolving', 'fee', 'interest', 'reporting', 'report', 'charged', 'feature', 'ter

In [ ]:
topic0 - Mortgages/loans
topic1 - Bank account services
topic2 - Credit card / Prepaid card
topic3 - Theft/Dispute reporting
topic4 - Others


In [101]:
result = nmf_model.transform(tfidf_vectors)
df['Topic'] = result.argmax(axis=1)

In [103]:
df.Topic.value_counts()

2    30807
1    13661
3    13274
0    10927
4     9644
Name: Topic, dtype: int64

In [104]:
df.head()

,big_corpus,Topic
0,Debt collection Credit card debt Attempts to c...,2
1,Debt collection Credit card debt Written notif...,2
2,Credit card or prepaid card General-purpose cr...,2
3,Mortgage Conventional home mortgage Trouble du...,3
4,Credit card or prepaid card General-purpose cr...,2


In [109]:
tfidf_vectors[3]

<1x13526 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [110]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [111]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors, df['Topic'], test_size=0.25, random_state=42)

In [113]:
logreg = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [114]:
logreg.score(X_test, y_test)

0.9748199601613974

In [115]:
y_pred = logreg.predict(X_test)

In [116]:
from sklearn.metrics import classification_report,confusion_matrix

In [118]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2858
           1       0.98      0.98      0.98      3416
           2       0.97      0.98      0.98      7705
           3       0.97      0.97      0.97      3266
           4       0.97      0.94      0.95      2334

    accuracy                           0.97     19579
   macro avg       0.98      0.97      0.97     19579
weighted avg       0.97      0.97      0.97     19579



In [119]:
print(confusion_matrix(y_test,y_pred))

[[2786    0   40   32    0]
 [   0 3351   33    2   30]
 [   4   43 7577   38   43]
 [   9    7   68 3178    4]
 [   2   19   94   25 2194]]


In [97]:
# from sklearn.decomposition import LatentDirichletAllocation
# lda = LatentDirichletAllocation(n_components=5)

In [98]:
# lda.fit(tfidf_vectors)

In [99]:
# for index,topic in enumerate(nmf_model.components_):
#     print(f'Topic {index} top 30 words are ')
#     print([tfidf.get_feature_names()[i] for i in topic.argsort()[::-1][:30]])
#     print('##########################')

In [94]:
topics

,topic_1,topic_2,topic_3,topic_4,topic_5
0,0.055597,0.777427,0.055758,0.055549,0.055669
1,0.028861,0.030686,0.030739,0.880297,0.029417
2,0.162361,0.033671,0.034063,0.735825,0.034080
3,0.057231,0.771302,0.057100,0.057113,0.057253
4,0.678200,0.048597,0.048533,0.050392,0.174278
...,...,...,...,...,...
78308,0.782197,0.054226,0.055137,0.054317,0.054122
78309,0.017429,0.017160,0.017115,0.930899,0.017397
78310,0.015717,0.015823,0.015959,0.936695,0.015805
78311,0.018465,0.018120,0.018127,0.926893,0.018395
